# Image postprocessing of time-resolved cavitation

Spectral analysis of cavitating flows from high speed images.
    
    @author Daniel Duke <daniel.duke@monash.edu>
    @copyright (c) 2020 LTRAC
    @license GPL-3.0+
    @version 0.0.1
    @date 16/07/2021
        __   ____________    ___    ______
       / /  /_  ____ __  \  /   |  / ____/
      / /    / /   / /_/ / / /| | / /
     / /___ / /   / _, _/ / ___ |/ /_________
    /_____//_/   /_/ |__\/_/  |_|\__________/

    Laboratory for Turbulence Research in Aerospace & Combustion (LTRAC)
    Monash University, Australia


In [1]:
from pySciCam.pySciCam import ImageSequence # https://github.com/djorlando24/pySciCam
import numpy as np
import glob, sys, os
import matplotlib.pyplot as plt
from scipy.interpolate import interp2d
import scipy.integrate, scipy.signal
from joblib import Parallel, delayed
%matplotlib notebook

In [2]:
# Define where images come from.
source_images = "/Users/dduke/Downloads/PHO_75_125_4001_CC_BS.tif"
#source_images = "/Users/dduke/Downloads/PHO_060_250_test5.tif"

## Load images
Crop and mask as required.

In [ ]:
I=ImageSequence(source_images, IO_threads=4, dtype=np.uint16)#, frames=(0,1024))

Reading /Users/dduke/Downloads/PHO_75_125_4001_CC_BS.tif
	Treating as multipage TIFF - estimated 12139 frames from file size
	PythonMagick thinks the bit depth is <class 'numpy.uint16'>
	Reading files into memory...
40 tasks on 4 processors


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  1.6min


In [ ]:
# Show some samples
fig=plt.figure()
plt.suptitle("Sample frames\n"+os.path.basename(source_images))
ax=fig.add_subplot(311)
h=ax.imshow(I.arr[0,...],cmap=plt.cm.gray)
plt.title("Frame 0")
plt.colorbar(h)
ax=fig.add_subplot(312)
h=ax.imshow(I.arr[int(I.N/2)-1,...],cmap=plt.cm.gray)
plt.title("Frame %i" % (int(I.N/2)-1))
plt.colorbar(h)
ax=fig.add_subplot(313)
h=ax.imshow(I.arr[I.N-1,...],cmap=plt.cm.gray)
plt.title("Frame %i" % (I.N-1))
plt.colorbar(h);

## Extract a timeseries for analysis
Apply limited spatial averaging over a cropped region, with the mean intensity removed.

In [ ]:
Ia = I.arr[:,:,0:128].astype(np.float32) # make float for averaging accurately
# Average over the wall-normal direction
Ia_spatial_avg = np.nanmean(Ia, axis=1)
fig=plt.figure()
ax=fig.add_subplot(111)
ax.plot(np.nanmean(Ia_spatial_avg,axis=-1),lw=1)
plt.xlabel('Frame')
plt.ylabel('Spatial Average intensity');

## Apply Welch's Method to the timeseries
Given a known sampling rate set below.

In [ ]:
from scipy.signal import welch
f, Pxx = welch(I.arr[:,:,0:50], fs=80e3,\
               window='hann', nperseg=1024, noverlap=256, nfft=None, detrend='constant',\
               return_onesided=True, scaling='density', axis=0, average='mean')

# Spectrum during steady state
frameRate = 5e3
Ib_steady = Ib[t0:t1,...] # cut out part of time domain
Ib_transv = scipy.integrate.trapz(np.nan_to_num(Ib_steady),x=y,axis=1) # integrate in y
Ib_F = np.zeros_like(Ib_transv) # make empty array
window = np.hanning(Ib_transv.shape[0])/scipy.integrate.trapz(np.hanning(Ib_transv.shape[0]))

# Set up function for one fft
def onefft(data, window):
    d = np.nan_to_num(data) # copy data
    d *= window # apply window from above
    d = np.hstack((d,np.zeros_like(d))) # zero pad.
    d -= np.mean(d) # remove mean
    F = np.abs(np.fft.fft(d))[:data.shape[0]] # one half of output.
    F /= scipy.integrate.trapz(F) # normalize
    F *= np.var(data) # make power spectral density
    F[F<=0] = 1e-20 # avoid taking log of zero
    return F

# Take spectrum over time at each X position
for i in range(Ib_F.shape[1]):
    Ib_F[:,i] = onefft(Ib_transv[:,i], window)
    
freq = np.linspace(0,0.5,Ib_F.shape[0])*frameRate # frequency axis
Ib_F_xavg = np.mean(Ib_F,axis=1) # average of all X positions.

# Take spectrum of the background reference pixel (in case light source was unsteady)
Fref = onefft(refp[t0:t1].astype(np.float)/np.nanmax(refp[t0:t1]), window)

# Take background spectrum away from the averaged spectrum
Ib_F_xavg -= Fref

# Plot transv. integrated space-time data
fig=plt.figure(figsize=(8,8))
ax=fig.add_subplot(311)
xx,tt=np.meshgrid(range(Ib_transv.shape[1]),range(Ib_transv.shape[0]))
h=ax.pcolormesh(xx,tt,Ib_transv)
plt.ylabel('Time [frame]')
plt.xlabel('X [px]')
plt.colorbar(h)
# Plot fft vs X
ax=fig.add_subplot(312)
xx,ff=np.meshgrid(range(Ib_F.shape[1]),np.log10(freq+1e-6))
h=ax.pcolormesh(xx,ff,np.log10(Ib_F),vmin=-2,vmax=np.log10(Ib_F_xavg.max()),cmap=plt.cm.gnuplot)
plt.ylim(np.log10(freq[1]),np.log10(freq[-1]))
# TODO: Need to make y labels 10^x instead of x...
plt.ylabel('Frequency [Hz]')
plt.xlabel('X [px]')
plt.colorbar(h)
# Plot X-averaged FFT.
ax=fig.add_subplot(313)
ax.loglog(freq,Ib_F_xavg,lw=1,c='k')
ax.loglog(freq,Fref,c='b',lw=1)
plt.xlabel(r'log$_{10}$(Frequency [Hz])')
plt.ylabel(r'PSD [$I^2 \cdot$ frame]')
plt.ylim(Ib_F_xavg[-1]*0.5,np.max(Ib_F_xavg)*2)
plt.subplots_adjust(hspace=.5);


